## Build a Jupyter-Redis Application with Docker Compose

Next we will build a Docker Compose application to be a Jupyter Notebook Server running in conjunction with a Redis server.

### Application Definition

We will use two files, `Dockerfile` and `docker-compose.yml` and a series of Python and html files  organized in the following directories:

    .
    ├── docker
    │   └── jupyter
    │       └── Dockerfile
    ├── docker-compose.yml
    ├── lib
    │   └── helper.py
    ├── main.py
    ├── static
    │   └── style.css
    └── templates
        └── iris.html

#### `docker-compose.yml`

The `docker-compose.yml` which defines the way the various services defining our application interact.

    version: '3'
    services:
        this_jupyter:
            build: docker/jupyter
            ports:
                - "8000:8888"
            volumes:
                - .:/home/jovyan
        this_redis:
           image: redis
           volumes:
               - redis_data:/data
        this_api:
            build: docker/jupyter
            ports:
              - "5000:5000"
            volumes:
                - .:/home/jovyan
            environment:
                - FLASK_APP=main.py
            entrypoint: ["flask","run", "--host=0.0.0.0"]
    volumes:
        redis_data:


The Compose file defines three services: `this_api`, `this_jupyter`, and `this_redis` and a volume, `redis_data`.

The `this_jupyter` service:

- Uses the `build:` keyword to define the service, meaning it will use a `Dockerfile` to build the image defining the service.
- Uses a `Dockerfile` contained in the `docker/jupyter`
- Attaches the local directory (`.`) to the (hopefully familiar) jupyter `WORKDIR`, `/home/jovyan`, as specified by the `volumes:` keyword.
- Forwards the exposed port `8888` to the port `8000` on the host machine, as specified by the `ports:` keyword.

The `this_redis` service:

- Uses the `image:` keyword to define the service, meaning it will pull an image from the Docker Hub registry.
- Uses the `redis` image.
- Attaches the volume `redis_data` to the `redis` `WORKDIR`, `/data`, as specified by the `volumes:` keyword.


The `this_api` service:

- Uses the `build:` keyword **and the exact same `Dockerfile`** as the `this_jupyter` service. 
- Attaches the local directory (`.`) to the (hopefully familiar) jupyter `WORKDIR`, `/home/jovyan`, as specified by the `volumes:` keyword.
- Forwards the exposed port `5000` to the port `5000` on the host machine, as specified by the `ports:` keyword.
- Defines an environment variable using the `environment:` keyword, `FLASK_APP` with the value `main.py`. This argument will be used by the `flask` command to look for the python file defining the behavior of the flask app.
- Defines an `entrypoint`. An entrypoint as the first command to be executed by the container when it launches. Here, we tell it to run `flask run --host-0.0.0.0`.

**$\square$ Note** The definition of every container defined in a `docker-compose.yml` file must begin with either the `image:` keyword or the `build:` keyword. 

#### `Dockerfile`

In `docker/jupyter` we include a `Dockerfile` to be used by the `this_jupyter` and `this_api` services. 

The `Dockerfile` consists of three instructions telling Docker how to build our image. 

    FROM jupyter/datascience-notebook
    RUN pip install redis flask
    EXPOSE 5000
    
`FROM jupyter/datascience-notebook` The file begins with a `FROM` instruction. This is mandatory for every `Dockerfile`. It tells Docker which image to use as the base image. Here, we use the `jupyter/datascience-notebook` image.

`RUN pip install redis flask` Next, we use `conda` to install libraries to be used by our services. The `redis` library will be used both by our `this_jupyter` and `this_api` services to interface with the `this_redis` service. The `flask` library will be used by the `this_api` service to build our API. 

`EXPOSE 5000` Finally, we expose the port `5000` from the Docker container. The container already has the port `8888` exposed and is how we have been connecting to the Jupyter server. Now we expose port `5000` so that we can connect to our API on this port. Note that we specified this connection in the `docker-compose.yml` file.  

#### API-application definition

    .
    ├── lib
    │   └── helper.py
    ├── main.py
    ├── static
    │   └── style.css
    └── templates
        └── iris.html


### Run Your Application with Compose

Now, you use the `docker-compose` command line tool to start the application. You will use the `-d` argument to specify that you wish to launch the application in detached mode.


```
$ docker-compose up -d
Creating network "05-redis_default" with the default driver
Creating volume "05-redis_redis_data" with default driver
Building this_jupyter
Step 1/3 : FROM jupyter/datascience-notebook
 ---> 03923014986d
Step 2/3 : RUN conda install --yes redis flask
 ---> Using cache
 ---> 7ced54b401ce
Step 3/3 : EXPOSE 5000
 ---> Using cache
 ---> 75d70dd58cab
Successfully built 75d70dd58cab
Successfully tagged 05-redis_this_jupyter:latest
Building this_api
Step 1/3 : FROM jupyter/datascience-notebook
 ---> 03923014986d
Step 2/3 : RUN conda install --yes redis flask
 ---> Using cache
 ---> 7ced54b401ce
Step 3/3 : EXPOSE 5000
 ---> Using cache
 ---> 75d70dd58cab
Successfully built 75d70dd58cab
Successfully tagged 05-redis_this_api:latest
Creating 05-redis_this_jupyter_1 ... done
Creating 05-redis_this_api_1     ... done
Creating 05-redis_this_redis_1   ... done
```

The `docker-compose up` command

1. Creates a network `05-redis_default` to which all of the services will connect.
1. Creates the volume `05-redis_redis_data` to hold the data used by the `this_redis` service.
1. Builds the `this_jupyter` service.
   - if this is the first time you are building the image you will see more output at each build step. 
   - Note that the build steps correspond to the three lines of our `Dockerfile`.
1. Verifies that the `redis` image is in the image cache (`Successfully tagged 05-redis_this_api:latest`). 
   - If the image is not in your image cache it will be pulled at this time and you will see more output.
1. Creates the three services.


In [3]:
import requests

request = requests.get('http://this_api:5000')

request.text 

'API is live!'